# Predict New Offer Success

There is a company, which prepared special new offer for their customers. They have a dataset which consists information about their clients and how they reacted to the previous offers.

## Objectives

Your goal is to help them with current offer to provide list of customers who will accept new offer and also senior management would like to know why company should send them this new offer.

## Dataset, columns business meaning

### Target feature
- `accepted` - flag related whether customer accepted or not the offer

### Input features
- `offer_class` - previous offer class. e.g:
- `name` - hashed customer name
- `gender` - customer geneder
- `age` - age in years
- `phone_calls` - number of phone conversations with client during last quarter
- `emails` - number of emails sent to client during last 6 months
- `customer_code` - customer code
- `salary` - customer estimated salary
- `offer_code` - previous offer code
- `customer_type` - type of customer
- `number` - serial number of customer device
- `offer_value` - previous offers total value
- `estimated_expenses` - estimated expenses

In [1]:
import pandas as pd
import numpy as np

# conda install pyarrow -c conda-forge
import pyarrow

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Load the data

In [2]:
df = pd.read_parquet('client_database.parquet', engine='pyarrow')

In [3]:
oryg_shape = df.shape
oryg_shape

(1309, 15)

- The dataset has 1309 rows (observations) and 15 columns (fetures).

In [4]:
df.head(6).T

,0,1,2,3,4,5
offer_class,Medium,Medium,Medium,Medium,Medium,Medium
accepted,yes,yes,no,no,no,yes
name,C7CBB5C5613449B,CFD09C0248BB417,A2A0DC541977473,9068458EB70D427,46F0CD19CF71429,060A000A1260427
gender,female,male,female,male,female,male
age,29,NaN,NaN,30,25,48
phone_calls,0,1,1,1,1,0
emails,0,2,3,2,2,0
customer_code,24160,113781,113781,113781,113781,19952
salary,21133.8,15155,15155,15155,15155,2655
offer_code,4AB,61A,DB4,9B6,191,62F


- There is a categorical feature `center` in the dataset which is not mentioned in the dataset description.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1308
Data columns (total 15 columns):
offer_class           1289 non-null object
accepted              1289 non-null object
name                  1289 non-null object
gender                1289 non-null object
age                   877 non-null float64
phone_calls           1286 non-null float64
emails                1287 non-null float64
customer_code         1265 non-null object
salary                1281 non-null float64
offer_code            1287 non-null object
customer_type         1287 non-null object
number                1280 non-null object
offer_value           1277 non-null float64
estimated_expenses    1286 non-null float64
center                1306 non-null object
dtypes: float64(6), object(9)
memory usage: 163.6+ KB


- There are 2 types of data: float64 and strings
- There are missing values in the dataset with `age` having most of them

In [6]:
print(df.groupby('accepted').size())

accepted
no     800
yes    489
dtype: int64


- The dataset is slightly imballanced.

## Numerical features

In [7]:
df_num = df.select_dtypes(include = ['float64'])
df_num.head()

,age,phone_calls,emails,salary,offer_value,estimated_expenses
0,29.0,0.0,0.0,21133.75,57.426571,4692.0
1,NaN,1.0,2.0,15155.00,141.639912,3164.0
2,NaN,1.0,3.0,15155.00,154.821130,1852.0
3,30.0,1.0,2.0,15155.00,106.256196,3753.0
4,25.0,1.0,2.0,15155.00,139.237147,2410.0


In [8]:
df_num.describe()

,age,phone_calls,emails,salary,offer_value,estimated_expenses
count,877.000000,1286.000000,1287.000000,1281.000000,1277.000000,1286.000000
mean,33.511973,0.497667,0.881896,3297.296011,128.693732,4576.947900
std,12.247058,1.041360,1.022320,5034.240427,57.677807,1909.458459
min,18.000000,0.000000,0.000000,0.000000,50.022619,1257.000000
25%,24.000000,0.000000,0.000000,789.580000,82.632085,2934.750000
50%,30.000000,0.000000,1.000000,1445.420000,118.645478,4544.000000
75%,41.000000,1.000000,1.000000,3127.500000,162.482961,6225.750000
max,80.000000,8.000000,10.000000,51232.920000,368.668534,7891.000000


In [9]:
df_num[df_num.phone_calls == 0].count()

age                   617
phone_calls           877
emails                877
salary                873
offer_value           868
estimated_expenses    876
dtype: int64

In [10]:
df_num[df_num.salary == 0].count()

age                    8
phone_calls           17
emails                17
salary                17
offer_value           17
estimated_expenses    17
dtype: int64

In [11]:
df_num[df_num.offer_value == 0].count()

age                   0
phone_calls           0
emails                0
salary                0
offer_value           0
estimated_expenses    0
dtype: int64

In [12]:
n = len(df_num)
for column in df_num:
    
    n_unique = len(df_num[column].unique())
    print('Feature: {}'.format(column))
    print('Enique values count: {}'.format(n_unique))
    print('Unique values ratio: {}'.format(n_unique / n))
    print('Missing values ratio: {}'.format(df_num[column].isnull().sum() / n))
    print('Median: {}'.format(df_num[column].median()))
    print('------------------------------------')
    
    print()

Feature: age
Enique values count: 73
Unique values ratio: 0.05576776165011459
Missing values ratio: 0.3300229182582124
Median: 30.0
------------------------------------

Feature: phone_calls
Enique values count: 8
Unique values ratio: 0.006111535523300229
Missing values ratio: 0.01757066462948816
Median: 0.0
------------------------------------

Feature: emails
Enique values count: 11
Unique values ratio: 0.008403361344537815
Missing values ratio: 0.01680672268907563
Median: 1.0
------------------------------------

Feature: salary
Enique values count: 282
Unique values ratio: 0.21543162719633308
Missing values ratio: 0.0213903743315508
Median: 1445.42
------------------------------------

Feature: offer_value
Enique values count: 1278
Unique values ratio: 0.9763177998472116
Missing values ratio: 0.024446142093200916
Median: 118.6454777
------------------------------------

Feature: estimated_expenses
Enique values count: 1186
Unique values ratio: 0.906035141329259
Missing values ratio

- There are 6 numerical features.
- The range of their values needs to be standardized.
- There are no kids represented in the dataset (min. age is 18). 25% of customers is 24 years old or below. Next 25% are customers of age 25-30. Next 25% of customers is up to 41 years old, and last 25% of customers are 42-80.
- There were 2-8 phone calls made only to 25% of customers. The rest got 0 or 1.
- 50% of customers got 0 or 1 email, 25% of customers got 2-10 emails.
- The min. salary is 0, median 1445.42 and max. 51232.
- The previous offers values range between 50 and 368 with the median of 119.
- Estimated expenses values range between 1257 and 7891 with the median of 4544.

### Categorical Features

In [13]:
#list(set(df.dtypes.tolist()))
#df_cat = df.select_dtypes(include = ['O'])

In [14]:
df_cat = df[list(set(df.columns) - set(df_num.columns))]
df_cat.head()

,number,accepted,customer_code,offer_class,name,center,customer_type,offer_code,gender
0,9E9FA,yes,24160,Medium,C7CBB5C5613449B,A,S,4AB,female
1,1E53D,yes,113781,Medium,CFD09C0248BB417,A,S,61A,male
2,1.36E+06,no,113781,Medium,A2A0DC541977473,A,S,DB4,female
3,F6529,no,113781,Medium,9068458EB70D427,B,S,9B6,male
4,E2FDF,no,113781,Medium,46F0CD19CF71429,A,S,191,female


In [15]:
#feature_unique_values = {}
n = len(df_cat)
for column in df_cat:
    n_unique = len(df_cat[column].unique())
    #feature_unique_values[column] = n_unique
    print('---------------------')
    print('Feature name: {}'.format(column))
    print('Unique values count (including NaN): {}'.format(n_unique))
    if n_unique <= 5:
        print('Most frequent values:')
        print(df_cat.groupby(column).size())
    print('Variability ratio: {}'.format(n_unique / n))
    print('Values count (not null): {}'.format(df_cat[column].notnull().sum()))
    print('Values count (null): {}'.format(df_cat[column].isnull().sum()))
    print('Missing values ratio: {}'.format(df_cat[column].isnull().sum() / n))
    print()

---------------------
Feature name: number
Unique values count (including NaN): 1280
Variability ratio: 0.9778456837280367
Values count (not null): 1280
Values count (null): 29
Missing values ratio: 0.02215431627196333

---------------------
Feature name: accepted
Unique values count (including NaN): 3
Most frequent values:
accepted
no     800
yes    489
dtype: int64
Variability ratio: 0.002291825821237586
Values count (not null): 1289
Values count (null): 20
Missing values ratio: 0.015278838808250574

---------------------
Feature name: customer_code
Unique values count (including NaN): 900
Variability ratio: 0.6875477463712758
Values count (not null): 1265
Values count (null): 44
Missing values ratio: 0.03361344537815126

---------------------
Feature name: offer_class
Unique values count (including NaN): 4
Most frequent values:
offer_class
High       272
Medium     322
Premium    695
dtype: int64
Variability ratio: 0.0030557677616501145
Values count (not null): 1289
Values count (nu

- I will remove `number`, `name`, `offer_code`, `customer_code` from features list as these are kind of ID
- ??? what to do with `customer_code`, how to encode 900 different values?

In [16]:
df.drop(columns=['number', 'name', 'offer_code', 'customer_code'], inplace=True)

## Missing values

In [17]:
def extract_columns_with_nans(df):
    columns_with_nans = df.isnull().sum()[df.isnull().sum() > 0]
    dic_nan = {'nans_count': columns_with_nans.values, 'percent': np.round(columns_with_nans.values * 100 / df.shape[0], 2)}
    df_nan = pd.DataFrame(data=dic_nan, index=columns_with_nans.index)
    df_nan.sort_values(by='percent', ascending=False, inplace=True)
    return df_nan

In [18]:
# Missing values by column
df_nan = extract_columns_with_nans(df)
df_nan

,nans_count,percent
age,432,33.00
offer_value,32,2.44
salary,28,2.14
phone_calls,23,1.76
estimated_expenses,23,1.76
emails,22,1.68
customer_type,22,1.68
offer_class,20,1.53
accepted,20,1.53
gender,20,1.53


In [19]:
# Missing values by row
missing_data_by_row = df.isnull().sum(axis=1)
missing_data_by_row.describe()

count    1309.000000
mean        0.492743
std         1.281836
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max        10.000000
dtype: float64

- 25% of rows have between 1 and 14 missing values.
- `age`, `offer_value`, `salary`, `phone_calls`, `estimated_expenses`, `emails` - NaNs will be filled with the median.
- `customer_code` - I will remove 44 rows with NaNs as I do not know how to deal with missing values and leaving them as additional category, would skew the original distribution.
- I will remove rows with `center` (3), `offer_code` (2) and `customer_type' (2)

In [20]:
# Replace age, offer_value NaN with the feature median
df[['age']] = df[['age']].fillna(df.median())
df[['offer_value']] = df[['offer_value']].fillna(df.median())
df[['salary']] = df[['salary']].fillna(df.median())
df[['phone_calls']] = df[['phone_calls']].fillna(df.median())
df[['estimated_expenses']] = df[['estimated_expenses']].fillna(df.median())
df[['emails']] = df[['emails']].fillna(df.median())

In [21]:
#df.loc[df['customer_code'].isna()]
#df.loc[df['column_name'].isin(some_values)]

In [22]:
df_nan = extract_columns_with_nans(df)
df_nan

,nans_count,percent
customer_type,22,1.68
offer_class,20,1.53
accepted,20,1.53
gender,20,1.53
center,3,0.23


In [23]:
# Abandon rows where there is NaN in the center, offer_code and customer_type columns
df = df[~df.center.isna()]
df = df[~df.customer_type.isna()]

In [24]:
df_nan = extract_columns_with_nans(df)
df_nan

,nans_count,percent


In [25]:
clean_shape = df.shape
clean_shape

(1284, 11)

In [26]:
# Initial & complete dataset comparison
num_removed_cols = oryg_shape[1] - clean_shape[1]
num_removed_cols_ratio = np.around(num_removed_cols * 100 / oryg_shape[1], 2)
num_removed_rows = oryg_shape[0] - clean_shape[0]
num_removed_rows_ratio = np.around(num_removed_rows * 100 / oryg_shape[0], 2)
print('Number (percent) of removed features: {} ({}%)'.format(num_removed_cols, num_removed_cols_ratio))
print('Number (percent) of removed observations (with NaNs): {} ({}%)'.format(num_removed_rows, num_removed_rows_ratio))

Number (percent) of removed features: 4 (26.67%)
Number (percent) of removed observations (with NaNs): 25 (1.91%)


In [27]:
df.head()

,offer_class,accepted,gender,age,phone_calls,emails,salary,customer_type,offer_value,estimated_expenses,center
0,Medium,yes,female,29.0,0.0,0.0,21133.75,S,57.426571,4692.0,A
1,Medium,yes,male,30.0,1.0,2.0,15155.00,S,141.639912,3164.0,A
2,Medium,no,female,30.0,1.0,3.0,15155.00,S,154.821130,1852.0,A
3,Medium,no,male,30.0,1.0,2.0,15155.00,S,106.256196,3753.0,B
4,Medium,no,female,25.0,1.0,2.0,15155.00,S,139.237147,2410.0,A


In [28]:
# Gather all data cleaning operations in one function
def clean_data(df):

    # Drop unneeded columns
    df.drop(columns=['number', 'name', 'offer_code', 'customer_code'], inplace=True)
    
    # Replace age, offer_value NaN with the feature median
    df[['age']] = df[['age']].fillna(df.median())
    df[['offer_value']] = df[['offer_value']].fillna(df.median())
    df[['salary']] = df[['salary']].fillna(df.median())
    df[['phone_calls']] = df[['phone_calls']].fillna(df.median())
    df[['estimated_expenses']] = df[['estimated_expenses']].fillna(df.median())
    df[['emails']] = df[['emails']].fillna(df.median())
    
    # Abandon rows where there is NaN in the center, offer_code and customer_type columns
    df = df[~df.center.isna()]
    df = df[~df.customer_type.isna()]
    
    return df

In [29]:
# Test clean_data function
shape1 = df.shape
df2 = pd.read_parquet('client_database.parquet', engine='pyarrow')
df2 = clean_data(df2)
shape2 = df2.shape

print(shape1, shape2)

(1284, 11) (1284, 11)


In [30]:
df2.head()

,offer_class,accepted,gender,age,phone_calls,emails,salary,customer_type,offer_value,estimated_expenses,center
0,Medium,yes,female,29.0,0.0,0.0,21133.75,S,57.426571,4692.0,A
1,Medium,yes,male,30.0,1.0,2.0,15155.00,S,141.639912,3164.0,A
2,Medium,no,female,30.0,1.0,3.0,15155.00,S,154.821130,1852.0,A
3,Medium,no,male,30.0,1.0,2.0,15155.00,S,106.256196,3753.0,B
4,Medium,no,female,25.0,1.0,2.0,15155.00,S,139.237147,2410.0,A


In [31]:
# Save clean dataset
filename = 'client_database_clean.parquet'
df.to_parquet(filename)

In [32]:
# Test
df3 = pd.read_parquet(filename, engine='pyarrow')
df3.shape

(1284, 11)

In [33]:
df3.head()

,offer_class,accepted,gender,age,phone_calls,emails,salary,customer_type,offer_value,estimated_expenses,center
0,Medium,yes,female,29.0,0.0,0.0,21133.75,S,57.426571,4692.0,A
1,Medium,yes,male,30.0,1.0,2.0,15155.00,S,141.639912,3164.0,A
2,Medium,no,female,30.0,1.0,3.0,15155.00,S,154.821130,1852.0,A
3,Medium,no,male,30.0,1.0,2.0,15155.00,S,106.256196,3753.0,B
4,Medium,no,female,25.0,1.0,2.0,15155.00,S,139.237147,2410.0,A
